# **상담음성 남자상담원만 추출**
 - text, wav 파일 경로 추출하여 csv 파일로 저장

In [ ]:
import os
os.getcwd()

In [ ]:
import os

# 폴더 경로를 지정합니다. 이 경로는 변경하셔야 합니다.
folder_path = "D:\\상담 음성\\Training"

# 디렉터리 내의 모든 아이템에 대해
for item_name in os.listdir(folder_path):
    # 해당 아이템이 디렉터리인 경우 출력합니다.
    if os.path.isdir(os.path.join(folder_path, item_name)):
        print(item_name)


In [ ]:
import os
import json
import pandas as pd

# 경로를 수정하는 함수
def modify_path(text_path):
    """Modify the given text path and return new audio and text paths."""
    base_text_dir = "D:\\상담 음성\\Training"
    base_audio_dir = "D:\\상담 음성\\Training"
    
    parts = text_path.split('/')
    D_folder = parts[1]
    J_folder = parts[2]
    S_folder = parts[3]
    file_name = parts[4]

    text_folder = f"[라벨]KtelSpeech_train_{D_folder}_label_0"
    new_text_path = os.path.join(base_text_dir, text_folder, D_folder, J_folder, S_folder, file_name)

    audio_folder = f"[원천]KtelSpeech_train_{D_folder}_wav_0"
    new_audio_path = os.path.join(base_audio_dir, audio_folder, D_folder, J_folder, S_folder, file_name.replace('.txt', '.wav'))
    
    return new_audio_path, new_text_path

def extract_consultant_dialogs(json_directory_path, csv_output_path):
    """Extract male consultant dialogs from a given directory and save as CSV."""
    columns = ['audioPath', 'textPath']
    result_df = pd.DataFrame(columns=columns)
    
    for filename in os.listdir(json_directory_path):
        if filename.endswith('.json'):
            json_file_path = os.path.join(json_directory_path, filename)
            with open(json_file_path, 'r', encoding='utf-8-sig') as file:
                json_content = json.load(file)

            speakers = json_content['dataSet']['typeInfo']['speakers']
            consultant = next((speaker for speaker in speakers if speaker['type'] == '상담원' and speaker['gender'] == '남'), None)
            
            if consultant:
                consultant_id = consultant['id']
                dialogs = json_content['dataSet']['dialogs']
                for dialog in dialogs:
                    if dialog['speaker'] == consultant_id:
                        audio_path, text_path = modify_path(dialog['textPath'])
                        result_df = result_df.append({'audioPath': audio_path, 'textPath': text_path}, ignore_index=True)
    
    if not result_df.empty:
        result_df.to_csv(csv_output_path, encoding='utf-8-sig', index=False)
        print("CSV 파일이 다음 위치에 저장되었습니다:", csv_output_path)
    else:
        print(f"{json_directory_path}에서 남성 상담원을 찾을 수 없습니다. CSV 파일이 생성되지 않았습니다.")

# 주요 실행 코드
base_directory = "D:\\상담 음성\\Training"
csv_directory = "C:\\Users\\user\\Jinwoo\\men_counselar_path"

if not os.path.exists(csv_directory):
    os.makedirs(csv_directory)

label_directories = [d for d in os.listdir(base_directory) if "[라벨]" in d]

for sub_directory in label_directories:
    D_folder = sub_directory.split('_')[2]
    J_folder = "J" + str(int(D_folder[1:]) + 31)
    
    # Check if the path is a directory
    path_to_check = os.path.join(base_directory, sub_directory, D_folder, J_folder)
    
    if os.path.isdir(path_to_check):
        for S_folder in os.listdir(path_to_check):
            if S_folder.startswith("S"):
                full_path = os.path.join(base_directory, sub_directory, D_folder, J_folder, S_folder)
                csv_output_path = os.path.join(csv_directory, f'consultant_dialogs_{D_folder}_{J_folder}_{S_folder}.csv')
                extract_consultant_dialogs(full_path, csv_output_path)
